# Photometric Color correction

## Import

In [ ]:
# Import some generally useful packages

import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [ ]:
from scipy import interpolate

In [ ]:
# Import the primary photometry classes from rubin_sim.photUtils

import rubin_sim.photUtils.Bandpass as Bandpass
import rubin_sim.photUtils.Sed as Sed

In [ ]:
from libradtranpy import  libsimulateVisible

## Configuration

In [ ]:
path_throughput="/sps/lsst/groups/auxtel/softs/github/desc/throughputs/baseline"

In [ ]:
atm_std_filename = "atmos_std.dat"

In [ ]:
filter_filenames = ["total_u.dat","total_g.dat","total_r.dat","total_i.dat","total_z.dat","total_y.dat" ]
filter_tagnames = ["u","g","r","i","z","y"]
filter_color = ["b","g","r","orange","grey","k"]
NFILT=len(filter_filenames)

In [ ]:
WLMIN=300.
WLMAX=1100.
WLBIN=1.
NWLBIN=int((WLMAX-WLMIN)/WLBIN)
WL=np.linspace(WLMIN,WLMAX,NWLBIN)

## Initialisation

In [ ]:
df_std=pd.DataFrame()

In [ ]:
df_std["wl"] = WL

### Standard atmosphere

In [ ]:
fullfilename=os.path.join(path_throughput,atm_std_filename)

In [ ]:
arr= np.loadtxt(fullfilename)

In [ ]:
f = interpolate.interp1d(x=arr[:,0], y=arr[:,1],fill_value="extrapolate")

In [ ]:
atm_std=f(WL)

In [ ]:
df_std["atm"] = atm_std

### Total filters

In [ ]:
FILTERWL = np.zeros((NWLBIN,3))
for index,filename in enumerate(filter_filenames):
    fullfilename=os.path.join(path_throughput,filename)
    arr= np.loadtxt(fullfilename)
    ff = interpolate.interp1d(x=arr[:,0], y=arr[:,1],fill_value="extrapolate")
    throughput1=ff(WL)
    throughput2=throughput1/atm_std
    tag1="tot_"+filter_tagnames[index]
    tag2="inst_"+filter_tagnames[index]
    df_std[tag1]= throughput1
    df_std[tag2]= throughput2
    indexes=np.where(throughput2>0.05)[0]
    FILTERWL[index,0]=WL[indexes.min()] 
    FILTERWL[index,1]=WL[indexes.max()] 
    lambdab=np.trapz(throughput2,WL)/np.trapz(throughput2/WL,WL)
    FILTERWL[index,2]=lambdab

In [ ]:
FILTERWL

In [ ]:
for index in range(NFILT):
    tag="inst_"+filter_tagnames[index]
    plt.plot(WL,df_std[tag].values,color=filter_color[index]) 
    plt.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    #plt.axvline(FILTERWL[index,0],color=filter_color[index],linestyle=":")

In [ ]:
for index in range(NFILT):
    tag="tot_"+filter_tagnames[index]
    plt.plot(WL,df_std[tag].values,color=filter_color[index]) 
    plt.axvline(FILTERWL[index,2],color=filter_color[index],linestyle=":")

## Functions

In [ ]:
def fII0(wl,s):
    return np.trapz(s/wl,wl)

In [ ]:
def fII1(wl,s,wlb):
    return np.trapz(s*(wl-wlb)/wl,wl)

In [ ]:
def fII2(wl,s,wlb):
    return np.trapz(s*(wl-wlb)**2/wl,wl)

In [ ]:
def fII10(wl,s,wlb):
    return fII1(wl,s,wlb)/fII0(wl,s)

In [ ]:
def fII20(wl,s,wlb):
    return fII2(wl,s,wlb)/fII0(wl,s)

In [ ]:
def CalculateIIntegrals(df_S,wlb):
    """
    """
    II0 = np.zeros(NFILT)
    II1 = np.zeros(NFILT)
    II2 = np.zeros(NFILT)
    II10 = np.zeros(NFILT)
    II20 = np.zeros(NFILT)
    
    for index in range(NFILT):
        tag="tot_"+filter_tagnames[index]
        wl=df_S["wl"].values
        S=df_S[tag].values
        II0[index] = fII0(wl,S)
        II1[index] = fII1(wl,S,wlb[index])
        II2[index] = fII2(wl,S,wlb[index])
        II10[index] = fII10(wl,S,wlb[index])
        II20[index] = fII20(wl,S,wlb[index])
        
    df_II = pd.DataFrame()
    df_II["II0"]=II0
    df_II["II1"]=II1
    df_II["II2"]=II2
    df_II["II10"]=II10
    df_II["II20"]=II20    
    return df_II
    

# Calculate Integrals for Standard atmosphere

In [ ]:
df_IIstd = CalculateIIntegrals(df_std,FILTERWL[:,2])

In [ ]:
df_IIstd 

# Airmass effect

In [ ]:
all_am = np.linspace(1,3,15)
pwv= 3
aer= 0.04
oz = 300

In [ ]:
am

In [ ]:
all_df_obs = []


for index,am in enumerate(all_am):
    path,thefile = libsimulateVisible.ProcessSimulationaer(am[0],pwv,oz,aer,0,prof_str='us',proc_str='sa',cloudext=0.0, FLAG_VERBOSE=False)
    data = np.loadtxt(os.path.join(path,thefile))
    f = interpolate.interp1d(x=data[:,0], y=data[:,1],fill_value="extrapolate")
    atm=f(WL)
    
    
    df_obs=pd.DataFrame()
    df_obs["wl"] = WL
    df_obs["atm"] = atm
    
    tag1="tot_"+filter_tagnames[index]
    tag2="inst_"+filter_tagnames[index]
    
    df_obs[tag1]= df_std[tag2].values*atm
    df_obs[tag2]= df_std[tag2].values
    
    all_df_obs.append(df_obs)
    